In [1]:
import sys
import numpy as np
from versioned_hdf5.staged_changes import StagedChangesArray

print(sys.modules["versioned_hdf5.staged_changes"])
print(sys.modules["versioned_hdf5.subchunk_map"])

<module 'versioned_hdf5.staged_changes' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/staged_changes.cpython-312-x86_64-linux-gnu.so'>
<module 'versioned_hdf5.subchunk_map' from '/home/crusaderky/github/versioned-hdf5/versioned_hdf5/subchunk_map.cpython-312-x86_64-linux-gnu.so'>


# StagedChangesArray demo

In [2]:
base = np.arange(64).reshape(8, 8)
base

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31],
       [32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47],
       [48, 49, 50, 51, 52, 53, 54, 55],
       [56, 57, 58, 59, 60, 61, 62, 63]])

In [3]:
s = StagedChangesArray.from_array(
    base,
    chunk_size=(2, 2),
)
s

StagedChangesArray<shape=(8, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 base slabs, 0 staged slabs>
slab_indices:
[[1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]
 [1 2 3 4]]
slab_offsets:
[[0 0 0 0]
 [2 2 2 2]
 [4 4 4 4]
 [6 6 6 6]]

In [4]:
s.slabs

[array([[0, 0],
        [0, 0]]),
 array([[ 0,  1],
        [ 8,  9],
        [16, 17],
        [24, 25],
        [32, 33],
        [40, 41],
        [48, 49],
        [56, 57]]),
 array([[ 2,  3],
        [10, 11],
        [18, 19],
        [26, 27],
        [34, 35],
        [42, 43],
        [50, 51],
        [58, 59]]),
 array([[ 4,  5],
        [12, 13],
        [20, 21],
        [28, 29],
        [36, 37],
        [44, 45],
        [52, 53],
        [60, 61]]),
 array([[ 6,  7],
        [14, 15],
        [22, 23],
        [30, 31],
        [38, 39],
        [46, 47],
        [54, 55],
        [62, 63]])]

In [5]:
np.testing.assert_array_equal(base, s[:])

### \_\_setitem\_\_()

In [6]:
s._setitem_plan((slice(2, 5), slice(3, 6)))

SetItemPlan<value_shape=(3, 3), value_view=[:, :], append 2 empty slabs, 7 slice transfers among 4 slab pairs, drop 0 slabs>
  slabs.append(np.empty((6, 2)))  # slabs[5]
  slabs.append(np.empty((2, 2)))  # slabs[6]
  # 2 transfers from slabs[2] to slabs[5]
  slabs[5][0:2, 0:2] = slabs[2][2:4, 0:2]
  slabs[5][2:4, 0:2] = slabs[2][4:6, 0:2]
  # 1 transfers from slabs[3] to slabs[5]
  slabs[5][4:6, 0:2] = slabs[3][4:6, 0:2]
  # 1 transfers from value to slabs[6]
  slabs[6][0:2, 0:2] = value[0:2, 1:3]
  # 3 transfers from value to slabs[5]
  slabs[5][0:2, 1:2] = value[0:2, 0:1]
  slabs[5][2:3, 1:2] = value[2:3, 0:1]
  slabs[5][4:5, 0:2] = value[2:3, 1:3]
slab_indices:
[[1 2 3 4]
 [1 5 6 4]
 [1 5 5 4]
 [1 2 3 4]]
slab_offsets:
[[0 0 0 0]
 [2 0 0 2]
 [4 2 4 4]
 [6 6 6 6]]

In [7]:
s[2:5, 3:6] = np.arange(9).reshape(3, 3) + 1000
s

StagedChangesArray<shape=(8, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 base slabs, 2 staged slabs>
slab_indices:
[[1 2 3 4]
 [1 5 6 4]
 [1 5 5 4]
 [1 2 3 4]]
slab_offsets:
[[0 0 0 0]
 [2 0 0 2]
 [4 2 4 4]
 [6 6 6 6]]

### changes()

In [8]:
s._changes_plan()

ChangesPlan<16 chunks>
  base[0:2, 0:2] = slabs[1][0:2, 0:2]
  base[0:2, 2:4] = slabs[2][0:2, 0:2]
  base[0:2, 4:6] = slabs[3][0:2, 0:2]
  base[0:2, 6:8] = slabs[4][0:2, 0:2]
  base[2:4, 0:2] = slabs[1][2:4, 0:2]
  base[2:4, 2:4] = slabs[5][0:2, 0:2]
  base[2:4, 4:6] = slabs[6][0:2, 0:2]
  base[2:4, 6:8] = slabs[4][2:4, 0:2]
  base[4:6, 0:2] = slabs[1][4:6, 0:2]
  base[4:6, 2:4] = slabs[5][2:4, 0:2]
  base[4:6, 4:6] = slabs[5][4:6, 0:2]
  base[4:6, 6:8] = slabs[4][4:6, 0:2]
  base[6:8, 0:2] = slabs[1][6:8, 0:2]
  base[6:8, 2:4] = slabs[2][6:8, 0:2]
  base[6:8, 4:6] = slabs[3][6:8, 0:2]
  base[6:8, 6:8] = slabs[4][6:8, 0:2]

In [9]:
list(s.changes())

[((slice(0, 2, 1), slice(0, 2, 1)), 1, (slice(0, 2, 1), slice(0, 2, 1))),
 ((slice(0, 2, 1), slice(2, 4, 1)), 2, (slice(0, 2, 1), slice(0, 2, 1))),
 ((slice(0, 2, 1), slice(4, 6, 1)), 3, (slice(0, 2, 1), slice(0, 2, 1))),
 ((slice(0, 2, 1), slice(6, 8, 1)), 4, (slice(0, 2, 1), slice(0, 2, 1))),
 ((slice(2, 4, 1), slice(0, 2, 1)), 1, (slice(2, 4, 1), slice(0, 2, 1))),
 ((slice(2, 4, 1), slice(2, 4, 1)),
  5,
  array([[  18, 1000],
         [  26, 1003]])),
 ((slice(2, 4, 1), slice(4, 6, 1)),
  6,
  array([[1001, 1002],
         [1004, 1005]])),
 ((slice(2, 4, 1), slice(6, 8, 1)), 4, (slice(2, 4, 1), slice(0, 2, 1))),
 ((slice(4, 6, 1), slice(0, 2, 1)), 1, (slice(4, 6, 1), slice(0, 2, 1))),
 ((slice(4, 6, 1), slice(2, 4, 1)),
  5,
  array([[  34, 1006],
         [  42,   43]])),
 ((slice(4, 6, 1), slice(4, 6, 1)),
  5,
  array([[1007, 1008],
         [  44,   45]])),
 ((slice(4, 6, 1), slice(6, 8, 1)), 4, (slice(4, 6, 1), slice(0, 2, 1))),
 ((slice(6, 8, 1), slice(0, 2, 1)), 1, (slice(6,

### \_\_getitem\_\_()

In [10]:
s._getitem_plan(())

GetItemPlan<output_shape=(8, 8), output_view=[:, :], 16 slice transfers among 6 slab pairs>
  # 4 transfers from slabs[1] to out
  out[0:2, 0:2] = slabs[1][0:2, 0:2]
  out[2:4, 0:2] = slabs[1][2:4, 0:2]
  out[4:6, 0:2] = slabs[1][4:6, 0:2]
  out[6:8, 0:2] = slabs[1][6:8, 0:2]
  # 2 transfers from slabs[2] to out
  out[0:2, 2:4] = slabs[2][0:2, 0:2]
  out[6:8, 2:4] = slabs[2][6:8, 0:2]
  # 2 transfers from slabs[3] to out
  out[0:2, 4:6] = slabs[3][0:2, 0:2]
  out[6:8, 4:6] = slabs[3][6:8, 0:2]
  # 4 transfers from slabs[4] to out
  out[0:2, 6:8] = slabs[4][0:2, 0:2]
  out[2:4, 6:8] = slabs[4][2:4, 0:2]
  out[4:6, 6:8] = slabs[4][4:6, 0:2]
  out[6:8, 6:8] = slabs[4][6:8, 0:2]
  # 3 transfers from slabs[5] to out
  out[2:4, 2:4] = slabs[5][0:2, 0:2]
  out[4:6, 2:4] = slabs[5][2:4, 0:2]
  out[4:6, 4:6] = slabs[5][4:6, 0:2]
  # 1 transfers from slabs[6] to out
  out[2:4, 4:6] = slabs[6][0:2, 0:2]

In [11]:
s[:]

array([[   0,    1,    2,    3,    4,    5,    6,    7],
       [   8,    9,   10,   11,   12,   13,   14,   15],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39],
       [  40,   41,   42,   43,   44,   45,   46,   47],
       [  48,   49,   50,   51,   52,   53,   54,   55],
       [  56,   57,   58,   59,   60,   61,   62,   63]])

In [12]:
# __getitem__ loads data transparently from base to output
# (no cache on read)
s

StagedChangesArray<shape=(8, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 base slabs, 2 staged slabs>
slab_indices:
[[1 2 3 4]
 [1 5 6 4]
 [1 5 5 4]
 [1 2 3 4]]
slab_offsets:
[[0 0 0 0]
 [2 0 0 2]
 [4 2 4 4]
 [6 6 6 6]]

### resize()
When enlarging, new chunks full of fill_value all point to `slabs[0][0:]`.

It may also be necessary to load partially resized chunks from base.

In [13]:
s._resize_plan((5, 9))

ResizePlan<append 0 empty slabs, 0 slice transfers among 0 slab pairs, drop 0 slabs>
slab_indices:
[[1 2 3 4 0]
 [1 5 6 4 0]
 [1 5 5 4 0]]
slab_offsets:
[[0 0 0 0 0]
 [2 0 0 2 0]
 [4 2 4 4 0]]

In [14]:
s.resize((5, 9))

In [15]:
s

StagedChangesArray<shape=(5, 9), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 base slabs, 2 staged slabs>
slab_indices:
[[1 2 3 4 0]
 [1 5 6 4 0]
 [1 5 5 4 0]]
slab_offsets:
[[0 0 0 0 0]
 [2 0 0 2 0]
 [4 2 4 4 0]]

In [16]:
s[:]

array([[   0,    1,    2,    3,    4,    5,    6,    7,    0],
       [   8,    9,   10,   11,   12,   13,   14,   15,    0],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23,    0],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31,    0],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39,    0]])

In [17]:
# __setitem__() encroaching on full chunks
# (wholly on chunk row 0, partially on chunk row 1)
s._setitem_plan((slice(0, 3), -1))

SetItemPlan<value_shape=(3,), value_view=[:, None], append 2 empty slabs, 3 slice transfers among 3 slab pairs, drop 0 slabs>
  slabs.append(np.empty((2, 2)))  # slabs[7]
  slabs.append(np.empty((2, 2)))  # slabs[8]
  # 1 transfers from slabs[0] to slabs[7]
  slabs[7][0:2, 0:1] = slabs[0][0:2, 0:1]
  # 1 transfers from value to slabs[8]
  slabs[8][0:2, 0:1] = value[0:2, 0:1]
  # 1 transfers from value to slabs[7]
  slabs[7][0:1, 0:1] = value[2:3, 0:1]
slab_indices:
[[1 2 3 4 8]
 [1 5 6 4 7]
 [1 5 5 4 0]]
slab_offsets:
[[0 0 0 0 0]
 [2 0 0 2 0]
 [4 2 4 4 0]]

In [18]:
s[:3, -1] = 2000

In [19]:
s

StagedChangesArray<shape=(5, 9), chunk_size=(2, 2), dtype=int64, fill_value=0, 4 base slabs, 4 staged slabs>
slab_indices:
[[1 2 3 4 8]
 [1 5 6 4 7]
 [1 5 5 4 0]]
slab_offsets:
[[0 0 0 0 0]
 [2 0 0 2 0]
 [4 2 4 4 0]]

In [20]:
s[:]

array([[   0,    1,    2,    3,    4,    5,    6,    7, 2000],
       [   8,    9,   10,   11,   12,   13,   14,   15, 2000],
       [  16,   17,   18, 1000, 1001, 1002,   22,   23, 2000],
       [  24,   25,   26, 1003, 1004, 1005,   30,   31,    0],
       [  32,   33,   34, 1006, 1007, 1008,   38,   39,    0]])

In [21]:
s._changes_plan()

ChangesPlan<14 chunks>
  base[0:2, 0:2] = slabs[1][0:2, 0:2]
  base[0:2, 2:4] = slabs[2][0:2, 0:2]
  base[0:2, 4:6] = slabs[3][0:2, 0:2]
  base[0:2, 6:8] = slabs[4][0:2, 0:2]
  base[0:2, 8:9] = slabs[8][0:2, 0:1]
  base[2:4, 0:2] = slabs[1][2:4, 0:2]
  base[2:4, 2:4] = slabs[5][0:2, 0:2]
  base[2:4, 4:6] = slabs[6][0:2, 0:2]
  base[2:4, 6:8] = slabs[4][2:4, 0:2]
  base[2:4, 8:9] = slabs[7][0:2, 0:1]
  base[4:5, 0:2] = slabs[1][4:5, 0:2]
  base[4:5, 2:4] = slabs[5][2:3, 0:2]
  base[4:5, 4:6] = slabs[5][4:5, 0:2]
  base[4:5, 6:8] = slabs[4][4:5, 0:2]

### astype()
`StagedChangesArray.astype()` loads all chunks, except those that are covered in fill_value.

Note that `slabs[1]` is no longer referenced after it.

In [22]:
s2 = s.astype(float)
s2

StagedChangesArray<shape=(5, 9), chunk_size=(2, 2), dtype=float64, fill_value=0.0, 4 base slabs, 5 staged slabs>
slab_indices:
[[9 9 9 9 8]
 [9 5 6 9 7]
 [9 5 5 9 0]]
slab_offsets:
[[ 0  6  8 10  0]
 [ 2  0  0 12  0]
 [ 4  2  4 14  0]]

In [23]:
s2._changes_plan()

ChangesPlan<14 chunks>
  base[0:2, 0:2] = slabs[9][0:2, 0:2]
  base[0:2, 2:4] = slabs[9][6:8, 0:2]
  base[0:2, 4:6] = slabs[9][8:10, 0:2]
  base[0:2, 6:8] = slabs[9][10:12, 0:2]
  base[0:2, 8:9] = slabs[8][0:2, 0:1]
  base[2:4, 0:2] = slabs[9][2:4, 0:2]
  base[2:4, 2:4] = slabs[5][0:2, 0:2]
  base[2:4, 4:6] = slabs[6][0:2, 0:2]
  base[2:4, 6:8] = slabs[9][12:14, 0:2]
  base[2:4, 8:9] = slabs[7][0:2, 0:1]
  base[4:5, 0:2] = slabs[9][4:5, 0:2]
  base[4:5, 2:4] = slabs[5][2:3, 0:2]
  base[4:5, 4:6] = slabs[5][4:5, 0:2]
  base[4:5, 6:8] = slabs[9][14:15, 0:2]

### Sparse arrays

In [24]:
s_sparse = StagedChangesArray.full(
    shape=(5, 8),
    chunk_size=(2, 2),
    dtype=int,
)
s_sparse[3, 2] = 1337
s_sparse

StagedChangesArray<shape=(5, 8), chunk_size=(2, 2), dtype=int64, fill_value=0, 0 base slabs, 1 staged slabs>
slab_indices:
[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]]
slab_offsets:
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]

In [25]:
s_sparse[:]

array([[   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0, 1337,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0]])

In [26]:
s_sparse._changes_plan()

ChangesPlan<1 chunks>
  base[2:4, 2:4] = slabs[1][0:2, 0:2]

In [27]:
list(s_sparse.changes())

[((slice(2, 4, 1), slice(2, 4, 1)),
  1,
  array([[   0,    0],
         [1337,    0]]))]

In [28]:
a = np.zeros((20_000, 20_000))
b = StagedChangesArray.from_array(a, chunk_size=(100, 100))

In [29]:
b

StagedChangesArray<shape=(20000, 20000), chunk_size=(100, 100), dtype=float64, fill_value=0.0, 200 base slabs, 0 staged slabs>
slab_indices:
[[  1   2   3 ... 198 199 200]
 [  1   2   3 ... 198 199 200]
 [  1   2   3 ... 198 199 200]
 ...
 [  1   2   3 ... 198 199 200]
 [  1   2   3 ... 198 199 200]
 [  1   2   3 ... 198 199 200]]
slab_offsets:
[[    0     0     0 ...     0     0     0]
 [  100   100   100 ...   100   100   100]
 [  200   200   200 ...   200   200   200]
 ...
 [19700 19700 19700 ... 19700 19700 19700]
 [19800 19800 19800 ... 19800 19800 19800]
 [19900 19900 19900 ... 19900 19900 19900]]